In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/49/35/41623ac3b581781169eed7f5fcd24bc114c774dc491fab5c05d8eb81af36/google_api_python_client-2.160.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib from https://files.pythonhosted.org/packages/1a/8e/22a28dfbd218033e4eeaf3a0533b2b54852b6530da0c0fe934f0cc494b29/google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for httplib2<1.dev0,>=0.19.0 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c123e4e84b8fbc4b84beb0e70d7c1608486/httplib2-0.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth!=2.24.0,!=2.25.0,<3.0.0.dev0,>=1.3

In [16]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          r"C:\Users\stijn.vanbalen\OneDrive - Colliers International\Downloads\client_secret_689523484519-p0fgpnln3en3hbdf48v7i09mg5le2msj.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
"""
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
    print("Labels:")
    for label in labels:
      print(label["name"])

except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")
*/"""


# Nu bijvoorbeeld alle berichten in je Inbox ophalen (max 100 in dit voorbeeld):
service = build("gmail", "v1", credentials=creds)
results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=100).execute()
messages = results.get('messages', [])
print(f"Aantal berichten gevonden: {len(messages)}")
# Voorbeeld: eerste bericht opvragen en printen
if messages:
    msg_id = messages[0]['id']
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    # Het 'payload' veld bevat de e-mail content
    payload = msg.get('payload', {})
    headers = payload.get('headers', [])
    # Een simpel voorbeeld: onderwerp (Subject) uit de headers halen
    subject = None
    for header in headers:
        if header['name'] == 'Subject':
            subject = header['value']
            break
    # Body kan in verschillende delen staan (multipart). Hier een eenvoudige extractie:
    parts = payload.get('parts', [])
    mail_body = ""
    for part in parts:
        if part['mimeType'] == 'text/plain':
            data = part['body']['data']
            mail_body = base64.urlsafe_b64decode(data).decode('utf-8')
    print("Onderwerp:", subject)
    print("Inhoud:\n", mail_body)

Aantal berichten gevonden: 100
Onderwerp: Dag buurtgenoten.
Inhoud:
 Nextdoor Je buren in Planetenwijk

 Dag buurtgenoten.
https://nextdoor.nl/p/QJ9nbwcymdZB?post=17592234070855&ct=Dkjngn4zxXJujDfWkjEcBAeNJuCJvQeO-dckvvdBBceaBs96jCWgzVCy-12x7W3w&ec=iD9EVf8Fqz_GipY9l9G7ig==&token=RlLa6mZBn9HjoWYIhAoGr9e1lSA6sSyyaCP-5bitJSJiMHQAfAhVyfij9u79f_b_mg48BfmuVeAp5_Y-3b_B5sl8NziL0zh7zca40Nh08k4%3D&auto_token=ZFbc-eA229SjoQUFG3YZEgs4Vh1Lrlvis5D58lDU0dDcPDhqFmkfai3tyGnw6wXplR4ZVc-ehCQXwqGxz4tMCin1EP9yR1nU08-9te8Upfo%3D

Lilette v. 
https://nextdoor.nl/profile/17592202341596?ct=Dkjngn4zxXJujDfWkjEcBAeNJuCJvQeO-dckvvdBBceaBs96jCWgzVCy-12x7W3w&ec=iD9EVf8Fqz_GipY9l9G7ig==&token=RlLa6mZBn9HjoWYIhAoGr9e1lSA6sSyyaCP-5bitJSJiMHQAfAhVyfij9u79f_b_mg48BfmuVeAp5_Y-3b_B5sl8NziL0zh7zca40Nh08k4%3D&auto_token=ZFbc-eA229SjoQUFG3YZEgs4Vh1Lrlvis5D58lDU0dDcPDhqFmkfai3tyGnw6wXplR4ZVc-ehCQXwqGxz4tMCin1EP9yR1nU08-9te8Upfo%3D

We zijn een paspoort van onze kat kwijt met al haar gegevens er in. Mogelijk
zijn we hem de afg

In [9]:
# Nu bijvoorbeeld alle berichten in je Inbox ophalen (max 100 in dit voorbeeld):
results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=100).execute()
messages = results.get('messages', [])
print(f"Aantal berichten gevonden: {len(messages)}")
# Voorbeeld: eerste bericht opvragen en printen
if messages:
    msg_id = messages[0]['id']
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    # Het 'payload' veld bevat de e-mail content
    payload = msg.get('payload', {})
    headers = payload.get('headers', [])
    # Een simpel voorbeeld: onderwerp (Subject) uit de headers halen
    subject = None
    for header in headers:
        if header['name'] == 'Subject':
            subject = header['value']
            break
    # Body kan in verschillende delen staan (multipart). Hier een eenvoudige extractie:
    parts = payload.get('parts', [])
    mail_body = ""
    for part in parts:
        if part['mimeType'] == 'text/plain':
            data = part['body']['data']
            mail_body = base64.urlsafe_b64decode(data).decode('utf-8')
    print("Onderwerp:", subject)
    print("Inhoud:\n", mail_body)

NameError: name 'service' is not defined

In [42]:
# Fetch the last 100 messages
results = service.users().messages().list(
    userId='me', 
    q='subject:(""follow-up) OR in:body(follow-up")', 
    maxResults=15
).execute()

messages = results.get('messages', [])
print(f"Aantal berichten gevonden: {len(messages)}")
all_content = ""

if messages:
    for message in messages:
        msg_id = message['id']
        subject, mail_body, date = get_message_content(service, msg_id)
        if subject:
            all_content += f"Subject: {subject}\n"
        if mail_body:
            all_content += f"Sent on: {date}\n\n"
        if mail_body:
            all_content += f"Body: {mail_body}\n\n"

# Export to a file
with open("email_content.txt", "w", encoding="utf-8") as f:
    f.write(all_content)

print("Data written to email_content.txt")


Aantal berichten gevonden: 15
Data written to email_content.txt


In [18]:
def get_message_content(service, msg_id):
  """Gets the subject and body of a message."""
  msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
  payload = msg.get('payload', {})
  headers = payload.get('headers', [])
  subject = None
  for header in headers:
      if header['name'] == 'Subject':
          subject = header['value']
          break
  parts = payload.get('parts', [])
  mail_body = ""
  for part in parts:
      if part['mimeType'] == 'text/plain':
          data = part['body']['data']
          mail_body = base64.urlsafe_b64decode(data).decode('utf-8')
          break
      if part['mimeType'] == 'multipart/alternative' or part['mimeType'] == 'multipart/related':
        for sub_part in part['parts']:
          if sub_part['mimeType'] == 'text/plain':
            data = sub_part['body']['data']
            mail_body = base64.urlsafe_b64decode(data).decode('utf-8')
            break
  if not mail_body:
    if payload['mimeType'] == 'text/plain':
        data = payload['body']['data']
        mail_body = base64.urlsafe_b64decode(data).decode('utf-8')
  return subject, mail_body

In [36]:
import base64
from email.utils import parsedate_to_datetime

def get_message_content(service, msg_id):
    # Verkrijg de e-mail met de message_id
    message = service.users().messages().get(userId='me', id=msg_id).execute()
    
    # Haal het onderwerp, de body en de datum van de e-mail op
    subject = None
    mail_body = None
    date_sent = None
    
    # Doorloop de headers om het onderwerp, de datum en andere gegevens te vinden
    for header in message['payload']['headers']:
        if header['name'] == 'Subject':
            subject = header['value']
        if header['name'] == 'Date':
            # Parse de 'Date' header naar een datetime object
            date_sent = parsedate_to_datetime(header['value'])
    
    # Haal de body op (mogelijk meerdere delen, bijv. tekst en HTML)
    if 'parts' in message['payload']:
        for part in message['payload']['parts']:
            if part['mimeType'] == 'text/plain':
                mail_body = part['body']['data']
                # Decodeer de body van base64
                mail_body = base64.urlsafe_b64decode(mail_body).decode('utf-8')
    
    return subject, mail_body, date_sent

if messages:
    all_content = ""  # Initialiseer de variabele voor het verzamelen van de inhoud
    for message in messages:
        msg_id = message['id']
        subject, mail_body, date_sent = get_message_content(service, msg_id)
        
        if subject:
            all_content += f"Subject: {subject}\n"
        if mail_body:
            all_content += f"Body: {mail_body}\n"
        if date_sent:
            all_content += f"Sent on: {date_sent.strftime('%Y-%m-%d %H:%M:%S')}\n\n"  # Formatteer de datum
